In [1]:
import re
from pymongo import MongoClient 

#預設就是自己
client = MongoClient('10.120.28.14',27017)
database = client['news']
collection =database['news']

#for post in collection.find():
    #print post['title']

# 給關鍵字進行搜尋+分群

In [ ]:
tag1 = "柯文哲"
tag2 = "雙城"

#查詢滿足多重條件的語法是{"$and":[{條件1},{條件2},{條件3},...]}
#另有$or, $not, $nor
#$all在pymongo已廢止, 因為與$and功能重複

for post in collection.find(
    {"$and":[                   
            {"content":{"$regex":tag1}},
            {"content":{"$regex":tag2}},
            {"date":{"$regex":"2016"}}
            ]},{"_id":0}):
    #print post['title']

In [4]:
import re
import jieba
from sklearn.feature_extraction.text import CountVectorizer
import io


jieba.set_dictionary('E:/text_mining/dict.txt')  #切換至中文繁體字庫
jieba.load_userdict('E:/text_mining/dict_cbdic.txt')#加入自建詞庫
jieba.load_userdict('E:/text_mining/dict_keyw_new.txt')  #


#給關鍵字----------------------------------------
tag1 = "柯文哲"
tag2 = "腳踏車"
#-----------------------------------------------
stoplist = []
title=[]
content =[]
contents=[]

with io.open('E:/text_mining/stop_words.txt','r',encoding = 'utf-8') as f:
    for line in f.readlines():
        stoplist.append(line.strip('\n'))

#把資料庫東西抓出來
for post in collection.find(
    {"$and":[                   
            {"content":{"$regex":tag1}},
            {"content":{"$regex":tag2}},
            #{"date":{"$regex":"2016"}},
            ]},{"_id":0}): 
    summary = post['content']
    content.append('/'.join(jieba.cut(summary)))
    for a in content:
        if not a in stoplist:
            contents.append(''.join(a))
            
    title.append(post['title'])
    
print type(contents)
print title[0]
print title[1]
print "---------------------------------"
#print contents[0]


# 準備計算字詞頻率
# get_feature_names() 可以看"特徵詞有哪些" (『一個字』及『標點符號』會自動被去除掉)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(content) #裡面要放list
#for i in vectorizer.get_feature_names():
    #print i, #印出特徵詞
print "----------------------"    

#特徵值總共有多少個
print "Feature_num",len(vectorizer.get_feature_names())

#文章數
print "文章數:",len(title)

#透過toarray()可看到詞頻矩陣的结果
#print X.toarray()


Building prefix dict from E:\text_mining\dict.txt ...
DEBUG:jieba:Building prefix dict from E:\text_mining\dict.txt ...
Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u144803eaa96482ea338b54ad8c7f9634.cache
DEBUG:jieba:Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u144803eaa96482ea338b54ad8c7f9634.cache
Loading model cost 0.524 seconds.
DEBUG:jieba:Loading model cost 0.524 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


<type 'list'>
喊沒錢 柯文哲：有鄉民沒網軍
比扯鈴還扯！柯P勞動局長名單 遭質疑早內定
---------------------------------
----------------------
Feature_num 15299
文章數: 270


In [14]:
#計算TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(X) # 塞入TF-IDF 函數進行計算 #对应的tfidf矩阵

#把權重矩陣存起來
weight = tfidf.toarray()

#整合"特徵值"，"count"，"weight"
a=0
#整合第某篇新聞的"特徵值"，"count"，"weight"
for i in vectorizer.get_feature_names():
    if int(X.toarray()[0][a]) > int(0):  #出現次數 > 0 再印出來
        print i ," ",X.toarray()[0][a]," Weight: ",weight[0][a]
    a=a+1

100   1  Weight:  0.0623675069871
20   1  Weight:  0.0387343232104
60   1  Weight:  0.0597228153329
一個   1  Weight:  0.0361371132494
一半   1  Weight:  0.0623675069871
一旦   1  Weight:  0.0857134408958
一時   1  Weight:  0.0724220534479
一級   1  Weight:  0.0857134408958
不宜   1  Weight:  0.0857134408958
不要   2  Weight:  0.074755334957
不贊成   1  Weight:  0.0857134408958
也都   1  Weight:  0.0675413068608
他沒   1  Weight:  0.0724220534479
他還   1  Weight:  0.0597228153329
以他   1  Weight:  0.0857134408958
估計   1  Weight:  0.0675413068608
但不   1  Weight:  0.0724220534479
住戶   1  Weight:  0.0724220534479
保存   1  Weight:  0.0724220534479
保留   1  Weight:  0.0675413068608
保護   1  Weight:  0.0638958291524
共同   1  Weight:  0.0523129605264
再回到   1  Weight:  0.0857134408958
出席   1  Weight:  0.0399191156769
力量   1  Weight:  0.0496682688721
動員   1  Weight:  0.0798319082545
動態   1  Weight:  0.075658894435
北北基   1  Weight:  0.075658894435
北市   2  Weight:  0.0682816529829
占掉   1  Weight:  0.0857134408958
參選人   2  

In [ ]:
#clustering-1
from sklearn import cluster
c = cluster.KMeans(n_clusters=6)
k_data = c.fit_predict(weight)  # weight 直接拿來用 或放 X 有多算一層TF_IDF 分群效果會好一點


In [30]:
#clustering-2
for idx, grp in enumerate(k_data):
    if grp == 5:
        print title[idx]

台南選情冷 旗幟、文宣大縮水
柯P「點亮板橋螢光夜騎」 笑稱「考察板橋路面」
劉屏專欄—柯文哲訪美的淚與笑
趕場座車遭追撞 柯嚇一跳
MG149風波 柯嗆︰連家配跟我講廉潔？
我騎腳踏車他開保時捷 柯嗆連：財產申報怎會一樣多
〈北部〉柯P騎5小時單車 來挺柯建銘
〈中部〉柯文哲騎單車輔選 彰化挺陳文彬
〈北部〉柯P車隊打氣 選將陪騎喊加油
〈台北都會〉柯言行惹惱綠營 預算協商恐不挺
〈中部〉柯P抵布袋 蔡易餘陪騎
〈台北都會〉柯文哲過河 夜騎挺綠營選將
〈台北都會〉李彥秀、吳思瑤進國會 出缺不補選
辯論後朱蔡支持度持平　柯文哲：別幻想一次會劇烈變動
北市二選區／挑戰唯一「綠地」　潘懷宗PK姚文智
姚文智不滿「柯潘」同台　柯P：Take It Easy會贏就好
「柯系」選得不錯？柯P沒給答案　綠色台灣胸章搶眼
人氣被賴神稱讚、在日本又被列隊歡迎　柯文哲：有點嚇到
柯文哲民調領先選情仍緊張　陳永興：還需要增加關鍵5%
柯文哲：選出馬總統代表我們水準很低，不要抱怨
相信「專業」放颱風假　柯P：準不準幾小時後就知道！
柯文哲勉勵打非典型選戰　將幫時代力量站台高歌但不助選
不出席國慶大典　柯文哲：躲起來不要惹麻煩
柯文哲騎車到新竹順勢推銷柯建銘：希望他當立法院長！
柯文哲好神！從台北騎單車到新竹　抵達時直呼：屁股好痛
狂踩單車243公里隔天照常上工　柯P：我有可怕恢復力
素人形象助攻,柯搶中間選票
齊攻東區,連柯擦邊交鋒步行掃街PK定點合照,對街「路過」
陳昇連21年跨年唱突沮喪,認定阿河「自殺」
柯文哲：宣布放颱風假時,心裡毛毛的
北市立委力抗7屆立委丁守中 民進黨吳思瑤當選 
周子瑜事件 柯文哲相煎何太急 
柯文哲板橋好像沒有自行車專用道 
日北高後佩琪幫按摩柯P沒這種奇怪動作 
雙城騎謀為柯文哲的體力與毅力按讚 
整理包歐吉桑堅持夢想 柯文哲自行車一日北高達陣 
柯文哲騎鐵馬2訪雲林 民眾讚神勇 
桃園立委黃金周末大晴天 選將趴趴走 
桃園立委柯文哲車隊停桃園 王寶萱送憨人毛巾 
柯文哲騎卡打車之旅 二度來台中拜訪 
台北立委選戰熱捉對廝殺 互控滿天飛 
台北立委六選區分析參選爆炸 老將新秀12搶1 
是否教蔡英文辯論 柯文哲民調沒掉很多 
柯文哲與潘懷宗到夜市 稱市政考察 
高市竹林公園 輪椅族奔馳打網球 
宋楚瑜曾邀當副手 柯文哲做好做滿 
竹市立委柯文哲挺柯建銘 